<a href="https://colab.research.google.com/github/japmanyakaur/Course-Recommendation-System/blob/main/Copy_of_COURSE_RECOMMENDATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTING LIBRARIES
import pandas as pd
import re
import numpy as np
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# LOADING THE DATASET
df = pd.read_csv(r"/content/udemy_courses.csv")


title_col = 'headline'
tags_col = 'objectives'
desc_col = 'curriculum'

# TEXT CLEANING
def simple_clean(text):
    text = str(text).lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    tokens = text.split()
    tokens = [t for t in tokens if t not in ENGLISH_STOP_WORDS]
    return ' '.join(tokens)


df['combined'] = df[title_col].fillna('') + ' ' + df[tags_col].fillna('') + ' ' + df[desc_col].fillna('')
df['cleaned'] = df['combined'].apply(simple_clean)

#  SENTENCE TRANSFORMER MODEL
model = SentenceTransformer('all-MiniLM-L6-v2')

#  BATCH ENCODING
def batch_encode(texts, batch_size=64):
    all_embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        emb = model.encode(batch, show_progress_bar=False)
        all_embeddings.extend(emb)
    return np.array(all_embeddings)

print("🔄 Encoding all course descriptions...")
embeddings = batch_encode(df['cleaned'].tolist())

#  RECOMMENDATION FUNCTION
def recommend_courses(user_input, df, embeddings, top_n=5):
    user_input_clean = simple_clean(user_input)
    user_embedding = model.encode([user_input_clean])
    similarities = cosine_similarity(user_embedding, embeddings)[0]
    top_indices = similarities.argsort()[::-1][:top_n]


    recommended = df.iloc[top_indices][['title', 'url']].copy()
    recommended['score'] = similarities[top_indices]

    print(f"\n Recommended Courses for: '{user_input}'\n")
    for idx, row in recommended.iterrows():
        print(f" {row['title']}  —  Score: {row['score']:.3f}  —  {row['url']}")

    return recommended.reset_index(drop=True)


#  EXAMPLE USE
query = "dsa in c++ and java for beginners"


results = recommend_courses(query, df, embeddings)

print("\n Top Course Recommendations for:", query)
print(results.to_string(index=False))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

🔄 Encoding all course descriptions...

📚 Recommended Courses for: 'dsa in c++ and java for beginners'

[✔] OCA Java SE 7 Programmer (OCAJP) Exam 1ZO-803 Part 1 of 3  —  Score: 0.566  —  https://www.udemy.com/course/oca-javase7-part1/
[✔] Java Programming Language for Beginners  —  Score: 0.556  —  https://www.udemy.com/course/core-java-programming-language-tutorial/
[✔] The Certification Course Of Java  —  Score: 0.546  —  https://www.udemy.com/course/learn-java-from-scratch-s/
[✔] Java Programming for Absolute Beginners  —  Score: 0.546  —  https://www.udemy.com/course/java-programming-for-absolute-beginners/
[✔] Java Programming Fundamentals  —  Score: 0.539  —  https://www.udemy.com/course/java-programming-fundamental/

📚 Top Course Recommendations for: dsa in c++ and java for beginners
                                                    title                                                                   url    score
OCA Java SE 7 Programmer (OCAJP) Exam 1ZO-803 Part 1 of 3     